In [1]:
using Revise
using GLMakie
include("EFITModule/EFIT.jl")
include("VoronoiGen.jl")
using Rotations


In [2]:
using ParallelStencil
using ParallelStencil.FiniteDifferences3D
using GLMakie

ParallelStencil.@reset_parallel_stencil()
USE_GPU = false
@static if USE_GPU
    @init_parallel_stencil(CUDA, Float32, 3)
else
    @init_parallel_stencil(Threads, Float32, 3)
end

In [3]:
nx = ny = nz = 100

nGrains = 1000


#Maximum sound speed in the model
cMax = 12000
cMin = 0.5*cMax
#Frequency, hz
f0=1e6
#Period
t0 = 1.00 / f0
#Maximum spatial increment
dx = cMin/(8*f0)
#Maximum time increment
dt = dx/(cMax*sqrt(3))

baseMat = Main.EFIT.AnisoMats["X6CrNi1811"]
c = baseMat.c
ρ = baseMat.ρ


7820.0f0

In [4]:
matGrid = fullRandomGen(nx,ny,nz,nGrains).+2
matGrid[:,:,1:3] .= 1
matGrid[:,:,(end-3):end] .= 1

matGrid[:,1:3,:] .= 1
matGrid[:,(end-3):end,:] .= 1

matGrid[1:3,:,:] .= 1
matGrid[(end-3):end,:,:] .= 1;


In [5]:
materials = Array{Main.EFIT.AnisoMat,1}()
push!(materials,Main.EFIT.AnisoMat(10,5,1))

for i in 1:nGrains+2
    mat = Main.EFIT.AnisoMat(ρ,  Main.EFIT.rotMatrix(c,rand(RotMatrix{3})))
    push!(materials, mat)
end

In [6]:
grid = Main.EFIT.EFITGrid(Int16.(matGrid),materials,dt,dx);


#Create a source function from which input can be taken
function source(t)
    v = exp(-((2*(t-2*t0)/(t0))^2))*sin(2*pi*f0*t)*0.1
    return Data.Number(v)
end
nframes = 6000
framerate = 30

tIterator = 0:grid.dt:grid.dt*nframes
fig,ax,plt = volume(Array(grid.vx),algorithm=:mip,colorrange = (0, 0.3),colormap=:curl, transparency=true)

#Step the simulation
function stepSim(t)
    println(t)
    #Isotropic step function
    Main.EFIT.SimStep!(grid)
    #Apply the source to the Z-direction
    @parallel (45:55,45:55,95:98) Main.EFIT.applySource!(grid.vx,grid.vy,grid.vz, Data.Number(0.0), Data.Number(0.0), (source(t)))

    #Update the plot
    plt.volume = sqrt.(grid.vx.^2 .+ grid.vy.^2 .+ grid.vz.^2)
end

record(stepSim, fig, "color_animation.mp4", tIterator; framerate = 30)


Creating grid of size 100, 100, 100


Initialized averaging dict, hashed 57233 material combinations

0.0
3.608439e-8
7.216878e-8
1.08253175e-7
1.4433756e-7
1.8042195e-7
2.1650635e-7
2.5259072e-7
2.8867512e-7
3.2475953e-7
3.608439e-7
3.969283e-7
4.330127e-7
4.6909707e-7
5.0518145e-7
5.412659e-7
5.7735025e-7
6.134346e-7
6.4951905e-7
6.856034e-7
7.216878e-7
7.577722e-7
7.938566e-7
8.29941e-7
8.660254e-7
9.021098e-7
9.3819415e-7
9.742786e-7
1.0103629e-6
1.0464473e-6
1.0825318e-6
1.1186161e-6
1.1547005e-6
1.1907849e-6
1.2268692e-6
1.2629537e-6
1.2990381e-6
1.3351224e-6
1.3712068e-6
1.4072913e-6
1.4433756e-6
1.47946e-6
1.5155445e-6
1.5516288e-6
1.5877132e-6
1.6237976e-6
1.659882e-6
1.6959664e-6
1.7320508e-6
1.7681351e-6
1.8042196e-6
1.8403039e-6
1.8763883e-6
1.9124727e-6
1.9485572e-6
1.9846416e-6
2.0207258e-6
2.0568102e-6
2.0928946e-6
2.128979e-6
2.1650635e-6
2.201148e-6
2.2372321e-6
2.2733166e-6
2.309401e-6
2.3454854e-6
2.3815699e-6
2.417654e-6
2.4537385e-6
2.489823e-6
2.5259073e-6
2.5619

LoadError: Screen not open!